In [1]:
from simtk.openmm.app import *
from simtk.openmm import *
from simtk.unit import *
from sys import stdout, exit, stderr
import numpy as np
from tqdm import tqdm

from tqdm import tqdm_notebook

you got numpy


In [2]:
input_name = 'solvated_equilibrated'
output_name = 'st_kras_0'

# Load up restart files

In [3]:

pdb = PDBFile('./output.pdb')
complex_state = XmlSerializer.deserialize(open(input_name+'_state.xml').read())
complex_system = XmlSerializer.deserialize(open(input_name+'_system.xml').read())



# Before doing anything: delete the protein restraint. 
It was the last force that was added so just remove the -1th force:

In [4]:
forces = complex_system.getForces()
print([i for i in forces])
complex_system.removeForce(len(forces)-1)

[<simtk.openmm.openmm.HarmonicBondForce; proxy of <Swig Object of type 'OpenMM::HarmonicBondForce *' at 0x7fa5a803edb0> >, <simtk.openmm.openmm.HarmonicAngleForce; proxy of <Swig Object of type 'OpenMM::HarmonicAngleForce *' at 0x7fa5a803ee40> >, <simtk.openmm.openmm.PeriodicTorsionForce; proxy of <Swig Object of type 'OpenMM::PeriodicTorsionForce *' at 0x7fa5a803ee70> >, <simtk.openmm.openmm.NonbondedForce; proxy of <Swig Object of type 'OpenMM::NonbondedForce *' at 0x7fa5a803eea0> >, <simtk.openmm.openmm.CustomGBForce; proxy of <Swig Object of type 'OpenMM::CustomGBForce *' at 0x7fa5a803eed0> >, <simtk.openmm.openmm.CMMotionRemover; proxy of <Swig Object of type 'OpenMM::CMMotionRemover *' at 0x7fa5a803ef00> >, <simtk.openmm.openmm.CustomNonbondedForce; proxy of <Swig Object of type 'OpenMM::CustomNonbondedForce *' at 0x7fa5a803ef30> >, <simtk.openmm.openmm.CustomCentroidBondForce; proxy of <Swig Object of type 'OpenMM::CustomCentroidBondForce *' at 0x7fa5a803ef60> >, <simtk.openmm.o

In [5]:
##add a carbon alpha restraint to keep the protein unfolding:                                                                                                                         
force1 = CustomExternalForce("k1*((x-x0)^2+(y-y0)^2+(z-z0)^2)")                                                                                                        
force1.addGlobalParameter("k1", 0.1*kilocalories_per_mole/angstroms**2)                                                                                                           
force1.addPerParticleParameter("x0")                                                                                                                                              
force1.addPerParticleParameter("y0")                                                                                                                                              
force1.addPerParticleParameter("z0")                                                                                                                                              
for i, z in zip(pdb.topology.atoms(), complex_state.getPositions()):                                                                                                                        
    if i.name=='CA':                                                                                                                                                              
        force1.addParticle(int(i.index), [z[0], z[1], z[2]])                                                                                                                      

complex_system.addForce(force1)              

8

Set up the simulation object, using the special grest integrator:

In [6]:

integrator = LangevinIntegrator(310*kelvin, 1/picosecond, 0.002*picoseconds)

platform = Platform.getPlatformByName('CUDA')
prop = {'CudaPrecision':'single', 'CudaDeviceIndex':'0'}
simulation = Simulation(pdb.topology, complex_system, integrator, platform, prop)

Reporter and load the state object (i.e. restart)

In [7]:
simulation.reporters.append(DCDReporter(output_name+'_traj.dcd', 125000))
simulation.reporters.append(StateDataReporter(output_name+'.log', 10000,
                                              step=True,
                                              time=True,
                                              potentialEnergy=True,
                                              temperature=True,
                                              totalSteps=5000000,
                                              speed=True))


print('loadstate')
simulation.loadState(input_name+'_state.xml')


loadstate


In [8]:
st = SimulatedTempering(simulation,
                            numTemperatures=30,
                            tempChangeInterval=5,
                            minTemperature=310*kelvin,
                            maxTemperature=360*kelvin,
                            reportInterval=500,
                            reportFile=(str(output_name)+'_temp.dat'))


0.0001
lewis


In [9]:
st.cutoff = 1e-8

In [ ]:
num_nano = 150
for i in tqdm_notebook(range(500*num_nano)):
    st.step(1000)
    if st._updateWeights==False:
        break
    print(st._weightUpdateFactor, 
          st._updateWeights, 
          str(np.around(i / (500*num_nano), 2)), 
          [np.around(i,1) for i in st._weights], end='\r')

In [ ]:
with open(output_name+'_system.xml', 'w') as f:
    f.write(
            XmlSerializer.serialize(
                complex_system
            )
    )

simulation.saveState(output_name+'_state.xml')
